<a href="https://colab.research.google.com/github/hiydavid/numerai-models/blob/main/numerai_tfdf_model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numerai TFDF Model * WIP *
---

TFDF model
* Tensorflow Decision Forest model


---
# Load data & libraries

In [ ]:
# set drive location
from google.colab import drive
drive.mount('/content/drive/', force_remount = False)
path = '/content/drive/MyDrive/Colab/numerai/'

Mounted at /content/drive/


In [ ]:
# install dependencies
!pip install -Uqq numerapi==2.4.5 tensorflow==2.5.1 tensorflow_decision_forests

     |████████████████████████████████| 454.4 MB 9.5 kB/s 
     |████████████████████████████████| 6.3 MB 40.7 MB/s 
     |████████████████████████████████| 4.0 MB 53.1 MB/s 
     |████████████████████████████████| 462 kB 85.7 MB/s 
     |████████████████████████████████| 1.2 MB 58.1 MB/s 


In [ ]:
# import dependencies
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# download the latest training dataset (takes around 30s)
training_data_url = "https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz" 
training_data = pd.read_csv(training_data_url)

--- 
# Convert dataframes to datasets

In [ ]:
# split training into training & validation
training_data, testing_data = train_test_split(training_data, test_size = 0.2)

training_data.shape, testing_data.shape #, tournament_data.shape

((401446, 314), (100362, 314))

In [ ]:
# find only the feature columns
feature_cols = training_data.columns[
    training_data.columns.str.startswith('feature')
    ].append(pd.Index(['target']))

In [ ]:
# convert features data to ds object
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    training_data[feature_cols], 
    label="target",
    task=tfdf.keras.Task.REGRESSION
)

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    testing_data[feature_cols], 
    label="target",
    task=tfdf.keras.Task.REGRESSION
)

In [ ]:
# create a TFDF model
tfdf.keras.get_all_models()

[tensorflow_decision_forests.keras.RandomForestModel,
 tensorflow_decision_forests.keras.GradientBoostedTreesModel,
 tensorflow_decision_forests.keras.CartModel]

---
# Train model

In [ ]:
# configure model
model = tfdf.keras.CartModel(task=tfdf.keras.Task.REGRESSION)

In [ ]:
# fit model
model.fit(x=train_ds)

6273/6273 [==============================] - 74s 11ms/step


In [ ]:
# evaluate the model
model.compile(metrics=["accuracy"])
print(model.evaluate(test_ds))

1569/1569 [==============================] - 19s 11ms/step - loss: 0.0000e+00 - accuracy: 0.0496
[0.0, 0.04961040988564491]


In [ ]:
# save model
model.save(path + "tfdf_cart_model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab/numerai/tfdf_cart_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab/numerai/tfdf_cart_model/assets
